# 數據前處理

In [ ]:
!conda activate ./venv

In [ ]:
import codecs
import os
import re
from PIL import Image

all_file_dir = 'data/'

train_image_dir = os.path.join(all_file_dir, "train") 
eval_image_dir = os.path.join(all_file_dir, "test")

for name in os.listdir(train_image_dir):    
    if ' ' in name:
        subpath = os.path.join(train_image_dir, name)
        for sub_name in os.listdir(subpath):
            new_sub_name = re.sub(' ', '_', sub_name)
            os.rename(subpath+'/'+sub_name, subpath+'/'+new_sub_name)
        new_name = re.sub(' ', '_', name)
        os.rename(train_image_dir+'/'+name, train_image_dir+'/'+new_name)

for name in os.listdir(eval_image_dir):
    if ' ' in name:
        subpath = os.path.join(eval_image_dir, name)
        for sub_name in os.listdir(subpath):
            new_sub_name = re.sub(' ', '_', sub_name)
            os.rename(subpath+'/'+sub_name, subpath+'/'+new_sub_name)
        new_name = re.sub(' ', '_', name)
        os.rename(eval_image_dir+'/'+name, eval_image_dir+'/'+new_name)

class_list = [c.replace(' ','') for c in os.listdir(all_file_dir + '/train') if os.path.isdir(os.path.join(all_file_dir+'/train', c)) and not c.startswith('.')]
class_list.sort()
print(class_list)
all_file_dir = 'data'

train_image_dir = os.path.join(all_file_dir, "train") 
eval_image_dir = os.path.join(all_file_dir, "test")


train_file = codecs.open(os.path.join(all_file_dir, "train.txt"), 'w')
eval_file = codecs.open(os.path.join(all_file_dir, "eval.txt"), 'w')

with codecs.open(os.path.join(all_file_dir, "label_list.txt"), "w") as label_list:
    label_id = 0
    for class_dir in class_list:
        label_list.write("{0}\t{1}\n".format(label_id, class_dir))
        image_path_train = os.path.join(train_image_dir, class_dir)
        image_path_eval = os.path.join(eval_image_dir, class_dir)
        for file in os.listdir(image_path_train):
            try:
                img = Image.open(os.path.join(image_path_train, file))
                train_file.write("{0}\t{1}\n".format(os.path.join(image_path_train, file), label_id))
            except Exception as e:
                pass
        for file in os.listdir(image_path_eval):
            try:
                img = Image.open(os.path.join(image_path_eval, file))
                eval_file.write("{0}\t{1}\n".format(os.path.join(image_path_eval, file), label_id))
            except Exception as e:
                pass
        label_id += 1
            
train_file.close()
eval_file.close()

# 訓練

In [ ]:
!python work/train_resnet.py

In [ ]:

!python work/train_spinalnet.py

# 驗證

In [ ]:

!python work/eval_resnet.py

In [ ]:

!python work/eval_spinalnet.py

# 驗證可視化

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def draw_train_eval_process(title,step,train_reward, eval_reward, label_train_reward, label_eval_reward, color_train, color_eval, label_y):
    plt.title(title, fontsize=24)
    plt.xlabel("epoch", fontsize=20)
    plt.ylabel(label_y, fontsize=20)
    plt.plot(step, train_reward,color=color_train,label=label_train_reward) 
    plt.plot(step, eval_reward,color=color_eval,label=label_eval_reward) 
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
## ResNet
result_resnet = np.load('result/resnet_result.npz')
resnet_train_rewards = result_resnet['all_train_rewards']
resnet_eval_rewards = result_resnet['all_eval_rewards']
steps = [i for i in range(len(resnet_train_rewards))]
draw_train_eval_process("ResNet training and eval",steps,resnet_train_rewards,resnet_eval_rewards, 
"ResNet trainning accuracy", "ResNet eval accuracy", "red","green", "resnet_train_eval_accuracy") 

In [ ]:
## SpinalNet
result_spinal = np.load('result/spinalnet_result.npz')
spinal_train_rewards = result_spinal['all_train_rewards']
spinal_eval_rewards = result_spinal['all_eval_rewards']
train_steps = [i for i in range(len(spinal_train_rewards))]
eval_steps = [i for i in range(len(spinal_eval_rewards))]
draw_train_eval_process("SpinalNet training and eval",steps,spinal_train_rewards,spinal_eval_rewards, 
"SpinalNet trainning accuracy", "SpinalNet eval accuracy", "red","green", "spinal_train_eval_accuracy") 